# Homework 8

## History of Data Science, Spring 2022

### Due Sunday, May 22nd at 11:59PM

Homeworks in DSC 90 are designed to have you engage with the week’s content. They will rely on you having completed the readings.

Homeworks in DSC 90 are distributed via Jupyter Notebooks. They contain a mix of reflection, mathematical, and programming questions. You will always submit your homework as a **PDF**, not a Jupyter Notebook. **For math and written questions, include your answer and all of your work in your PDF. For coding questions, include screenshots of your code and all outputs in your PDF.**

We encourage you to discuss the readings and questions with others in the course, but all work must be your own. Remember to use Slack if you need guidance!

In [1]:
import pandas as pd
import numpy as np
from scipy.special import comb

from polynomial import *

## Question 1 – Method of Finite Differences

In this question, you will implement the method of finite differences to evaluate a third-degree polynomial on a range of inputs. This is what Charles Babbage's Difference Engine was designed to do.

<img src='https://cdn.britannica.com/10/23610-050-6E34CF6B/portion-Difference-Engine-Charles-Babbage-logarithm-tables-1832.jpg' width=300>

**Before proceeding, it's a good idea to re-watch the Lecture 8 recording, in which we evaluated a polynomial by writing out a table that followed the Difference Engine algorithm.**

In this question, you will interface with the `Polynomial` class. (Look at `hw07/polynomial.py` if you're curious as to how it works.)

Below, we create an example `Polynomial` object. To instantiate a `Polynomial`, you must provide a list of its coefficients. The first element in the list will be the constant term, the second element in the list will be the coefficient on $x$, the third element in the list will be the coefficient on $x^2$, and so on. (Note that this may be the reverse of what you'd expect.)

In [2]:
p = Polynomial([1, -2, 3, 4])
p

$1-2x+3x^2+4x^3$

`Polynomial`s have two attributes – `coefs` and `deg`:

In [3]:
p.coefs # The coefficients of the polynomial (from lowest degree to highest degree)

array([ 1, -2,  3,  4])

In [4]:
p.deg # The degree of the polynomial

3

They also have a method, `as_function`, which is a higher-order function (which means it returns another function).

In [5]:
p_f = p.as_function()
p_f

<function polynomial.Polynomial.as_function.<locals>.f(x)>

`p_f` is now a regular Python function, that can be used to evaluate the polynomial $p(x) = 1 - 2x + 3x^2 + 4x^3$ for any input. For example, $p(1)$ and $p(-4)$ are given by:

In [6]:
p_f(1)

6

In [7]:
p_f(-4)

-199

**Your Job:** Complete the implementation of the function `eval_mod`, which takes in:
- `p`, a `Polynomial` of degree 3 (meaning the polynomial is of the form $a_0 + a_1 x + a_2 x^2 + a_3 x^3$), and
- `arr`, an array consisting of evenly-spaced numbers

`eval_mod` should return an array that has the same length as `arr`, containing the output of `p` when evaluated on each element of `arr`. Example behavior is shown below.

```py
>>> p_test = Polynomial([5, -4, 2, -2])
>>> arr_test = np.array([-2, 0, 2, 4, 6, 8, 10, 12])
>>> eval_mod(p_test, arr_test)
array([   37.,     5.,   -11.,  -107.,  -379.,  -923., -1835., -3211.])
```

To make sure that you understand the above output, let $f(x) = 5 - 4x + 2x^2 - 2x^3$. Then, $f(-2) = 37$, $f(0) = 5$, $f(2) = -11$, and so on.

Now, this can be done in just two lines:
```py
def not_eval_mod(p, arr):
    p_f = p.as_function()
    return np.array([p_f(x) for x in arr])
```

However, this involves calling `p_f` once for each element of `arr`, and involves doing many multiplications and exponentiations. The whole goal of the method of finite differences is to compute the necessary outputs while using addition only, to the extent possible.

**`eval_mod` should use the method of differences** (after all, that's what "mod" stands for). Most of the implementation of `eval_mod` is done for you; much of your job is to understand the steps that have already been completed for you and see how they relate to the steps from lecture. **The amount of code you need to write is ~5 lines.**

In [8]:
def eval_mod(p, arr):
    # Compute the first four outputs manually
    p_f = p.as_function()
    f0 = p_f(arr[0])
    f1 = p_f(arr[1])
    f2 = p_f(arr[2])
    f3 = p_f(arr[3])
    
    # Below this line, p_f cannot be used!
    initial_values = np.array([f0, f1, f2, f3])
    
    # Find the first differences
    diffs = np.diff(initial_values)
    
    # Find the second differences
    diffs2 = np.diff(diffs)
    
    # Find the third differences
    diffs3 = np.diff(diffs2)
    
    # Find the first element in each of the differences columns – these are what we need to repeatedly add by
    d1 = diffs[0]
    d2 = diffs2[0]
    d3 = diffs3[0]
    
    outputs = np.array([]) # Do not change this line
    
    # YOUR CODE HERE – do not use p_f or .as_function() below!
    for _ in np.arange(len(arr)):
        outputs = np.append(outputs, f0)
        f0 = f0 + d1
        d1 = d1 + d2
        d2 = d2 + d3
    # YOUR CODE HERE
    
    return outputs

After you've completed `eval_mod`, run the cells below.

In [9]:
p_test = Polynomial([5, -4, 2, -2])
arr_test = np.array([-2, 0, 2, 4, 6, 8, 10, 12])
eval_mod(p_test, arr_test)

array([   37.,     5.,   -11.,  -107.,  -379.,  -923., -1835., -3211.])

The output above should be the same as the output below (though it is fine if you see trailing `.` signs above):

In [10]:
np.array([p_test.as_function()(i) for i in arr_test])

array([   37,     5,   -11,  -107,  -379,  -923, -1835, -3211])

Let's look at one more test case:

In [11]:
p_other = Polynomial([1, 0, 1, 3])
arr_other = np.arange(0, 100)
output_other = eval_mod(p_other, arr_other)
output_other[:7]

array([  1.,   5.,  29.,  91., 209., 401., 685.])

The outputs you see above should match the outputs you see below:

In [12]:
output_other_direct = np.array([p_other.as_function()(i) for i in arr_other])
output_other_direct[:7]

array([  1,   5,  29,  91, 209, 401, 685])

In addition, the following cell should evaluate to `True`:

In [13]:
np.allclose(output_other, output_other_direct)

True

**In your PDF writeup, include a screenshot of all of the code you wrote plus the outputs of all of the above code cells.**

## Question 2 – Bernoulli Numbers

Recall from lecture, Ada Lovelace wrote a program for the Analytical Engine that specified how to compute the sequence of Bernoulli numbers.

In this question, you too will write a function that returns the sequence of Bernoulli numbers.

### Question 2.1

The Bernoulli Numbers are a sequence of numbers that are relevant in computing sums of the form $1^m + 2^m + 3^m + ... + n^m$. (They are named for Jakob Bernoulli, the mathematician who also first posed the Law of Large Numbers.) The first few numbers in the sequence are shown below:

$$B_0 = 1, B_1 = -\frac{1}{2}, B_2 = \frac{1}{6}, B_3 = 0, B_4 = -\frac{1}{30}, B_5 = 0, B_6 = \frac{1}{42}, B_7 = 0$$

Read Pages 3-7 of [The Bernoulli Numbers: A Brief Primer](https://www.whitman.edu/documents/Academics/Mathematics/2019/Larson-Balof.pdf) to gain an understanding of where the numbers come from.

There's nothing to answer in this part of the question, but you **need** to do this reading in order to proceed.

### Question 2.2

On Page 10 of the above reading, you'll see a proposition that says that the Bernoulli numbers satisfy the relation

$$B_0 = 1 \text{ and } \sum_{k=0}^{n-1} {n \choose k} B_k = 0, \text{ for } n > 1$$

With a little bit of rearranging, we can use the above relation to express $B_{n-1}$ in terms of the Bernoulli numbers before it, namely $B_0, B_1, B_2, ..., B_{n-2}$:

$$
\begin{align*}
\sum_{k=0}^{n-1} {n \choose k} B_k &= 0 \\
\sum_{k=0}^{n-2} {n \choose k} B_k + {n \choose n - 1} B_{n-1} &= 0 \\
{n \choose n - 1} B_{n-1} &= -\sum_{k=0}^{n-2} {n \choose k} B_k \\
\implies B_{n-1} &= \boxed{-\frac{1}{n}\sum_{k=0}^{n-2} {n \choose k} B_k}
\end{align*}
$$

(If you're not sure how we proceeded from Step 3 to Step 4, recall that ${n \choose n-1} = n$.)

Let's use this **recurrence relation** to find $B_1$ and $B_2$, starting with just $B_0 = 1$.

To find $B_1$, $n-1$ must be 1 and so $n$ must be 2:

$$B_1 = -\frac{1}{2}\sum_{k=0}^{0} {2 \choose k} B_k = -\frac{1}{2} {2 \choose 0} B_0 = -\frac{1}{2} \cdot 1 \cdot 1 = -\frac{1}{2}$$

So $B_1 = -\frac{1}{2}$. To find $B_2$, $n-1$ must be 2 and so $n$ must be 3:

$$B_2 = -\frac{1}{3} \sum_{k = 0}^1 {3 \choose k} B_k = -\frac{1}{3} \left[ {3 \choose 0} B_0 + {3 \choose 1} B_1 \right] = -\frac{1}{3} \left[ 1 - 3 \cdot \frac{1}{2} \right] = \frac{1}{6}$$

So $B_2 = \frac{1}{6}$.

Note that in the calculation of $B_2$, we used the values of $B_0$ and $B_1$ that we found earlier.

**Your Job:** Complete the implementation of the function `bernoulli_sequence(N)`, which takes in an integer `N` and returns an array containing first `N+1` Bernoulli numbers (i.e. $B_0, B_1, B_2, ..., B_N$).

Some guidance:
- While it is technically possible to accomplish this by creating a recursive helper function, we highly advise against it, as it will lead to many unnecessary computations. Instead, a good strategy is to keep track of all of the Bernoulli numbers computed so far in an array `nums` that you can always refer back to.
- You'll need a nested `for`-loop. Choose the ranges for your `for`-loops very carefully. (What was the smallest value of $n$ that we looked at above? For each $n$, what values of $k$ did we look at above?)
- Recall from Homework 5, `comb(n, k)` evaluates ${n \choose k}$. We've already run `from scipy.special import comb` for you, so you can (and will need to) use the `comb` function in your code.

In [14]:
def bernoulli_sequence(N):
    nums = np.array([1]) # Since B_0 = 1
    # YOUR CODE HERE
    for n in np.arange(2, N + 2):
        total = 0
        for k in np.arange(0, n - 1):
            total += comb(n, k) * nums[k]
        total *= -1 / n
        nums = np.append(nums, total)
    # YOUR CODE HERE
    return nums

After you've completed `bernoulli_sequence`, run the cells below.

In [15]:
bernoulli_19 = bernoulli_sequence(18)
for i, b in enumerate(bernoulli_19):
    print(f'B_{i} = {np.round(b, 6)}')

B_0 = 1.0
B_1 = -0.5
B_2 = 0.166667
B_3 = -0.0
B_4 = -0.033333
B_5 = -0.0
B_6 = 0.02381
B_7 = 0.0
B_8 = -0.033333
B_9 = 0.0
B_10 = 0.075758
B_11 = -0.0
B_12 = -0.253114
B_13 = 0.0
B_14 = 1.166667
B_15 = -0.0
B_16 = -7.092157
B_17 = -0.0
B_18 = 54.971178


The values you see above should match the values on Page 9 of [the above document](https://www.whitman.edu/documents/Academics/Mathematics/2019/Larson-Balof.pdf). The following cell should evaluate to `True`.

In [16]:
correct_19 = np.array([1, -1/2, 1/6, 0, -1/30, 0, 1/42, 0, -1/30, 0, 5/66, 0, 
                       -691/2730, 0, 7/6, 0, -3617/510, 0, 43867/798])

np.allclose(bernoulli_19, correct_19)

True

**In your PDF writeup, include a screenshot of all of the code you wrote plus the outputs of all of the above code cells.**

### Question 2.3

Page 7 of [the aforementioned document](https://www.whitman.edu/documents/Academics/Mathematics/2019/Larson-Balof.pdf) contains the following formula:

$$S_m(n-1) = \frac{1}{m+1} \sum_{k=0}^m {m + 1 \choose k} B_k n^{m - k + 1}$$

Note that $S_m(n-1)$ means $1^m + 2^m + 3^m + ... + (n-1)^m$.

Below, complete the implementation of the function `sum_with_bernoulli`, which takes in a value of `m` and a value of `n` and returns the value of the sum $1^m + 2^m + 3^m + ... + (n-1)^m + n^m$ using the above formula.

Some guidance:
- Your function should include $n^m$, it should not stop at $(n-1)^m$. (One way to do this is to replace the $n$ on the right side of the above equation with $n+1$.)
- You will have to call `bernoulli_sequence` in your implementation.

In [17]:
def sum_with_bernoulli(m, n):
    # YOUR CODE HERE
    nums = bernoulli_sequence(m)
    total = 0
    for k in range(m + 1):
        total += comb(m + 1, k) * nums[k] * ((n + 1) ** (m - k + 1))
    return total / (m + 1)
    # YOUR CODE HERE

After you've completed `sum_with_bernoulli`, run the cells below.

In [18]:
# Should evaluate to 5050
sum_with_bernoulli(1, 100)

5050.0

In [19]:
# Should evaluate to 42925
sum_with_bernoulli(2, 50)

42925.0

In [20]:
# Should evaluate to 5050^2 = 25502500
sum_with_bernoulli(3, 100)

25502500.0

To help test out your function's behavior on more examples, we've defined `sum_direct`, which takes in the same arguments as `sum_with_bernoulli` and returns the same result, but without using Bernoulli numbers.

(If your `sum_with_bernoulli` looks like `sum_direct` below, you didn't implement it correctly – `sum_with_bernoulli` **must** involve the Bernoulli numbers, and will involve a `for`-loop!)

In [21]:
def sum_direct(m, n):
    return np.sum(np.arange(1, n + 1) ** m)

Repeatedly change `m` and `n` below and run the cell. You should always see `True`, as long as `m` and `n` are relatively small (for larger `m` and `n`, numerical imprecision issues may cause `bernoulli_sequence` and hence `sum_with_bernoulli` to yield incorrect answers).

In [22]:
m, n = 5, 45
np.isclose(sum_with_bernoulli(m, n), sum_direct(m, n))

True

**In your PDF writeup, include a screenshot of all of the code you wrote plus the outputs of all of the above code cells.**

## Question 3 – Makeup Question

If you:
- didn't originally answer Question 3 on Homework 5 (the optional make-up problem), **and**
- you've missed a lecture or didn't receive full credit on an earlier homework, 

you can answer Question 3 from Homework 5 and it will "make up" a missed lecture or homework. Note that this is separate from Question 2.3 above – if you answer both, you can make up 2 missed lectures or homeworks.

(Submit it as the answer to Question 3 for this homework.)

Homework 5 can be found [here](https://historyofdsc.com/resources/weeks/week05/).